In [1]:
import sys
import os
from easydict import EasyDict

import h5py

import json
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

from torch2trt import torch2trt
from torch2trt import TRTModule

from alphapose.models import builder
from alphapose.utils.config import update_config
from alphapose.utils.transforms import get_func_heatmap_to_coord
from alphapose.utils.bbox import (_box_to_center_scale, _center_scale_to_box)
from alphapose.utils.transforms import (get_affine_transform, im_to_torch)
from detector.yolo4_csp.ScaledYOLOv4.utils.torch_utils import time_synchronized

In [2]:
sys.path.append('./detector/')
sys.path.append('./detector/yolo4_csp/ScaledYOLOv4/')
sys.path.append('./detector/yolo/')

In [3]:
from detector.yolo4_api import YOLO4Detector
from detector.yolo4trt_api import YOLO4TRTDetector
from detector.yolo4_cfg import cfg as yolo_cfg
from detector.yolo4_csp.preprocess import prep_image

In [4]:
class Transform(object):
    """Generation of cropped input person and pose heatmaps from SimplePose.

    Parameters
    ----------
    input_size: tuple
        Input image size, as (height, width).
    output_size: tuple
        Heatmap size, as (height, width).
    """

    def __init__(self,input_size, output_size, gpu_device='cuda'):
        self._gpu_device = gpu_device

        self._input_size = input_size
        self._heatmap_size = output_size

        self._aspect_ratio = float(input_size[1]) / input_size[0]  # w / h
        
    def preprocess(self, src, bbox):
        x, y, w, h = bbox
        center, scale = _box_to_center_scale(
            x, y, w, h, self._aspect_ratio)
        scale = scale * 1.0

        input_size = self._input_size
        inp_h, inp_w = input_size

        trans = get_affine_transform(center, scale, 0, [inp_w, inp_h])
        img = cv2.warpAffine(src, trans, (int(inp_w), int(inp_h)), flags=cv2.INTER_LINEAR)
        bbox = _center_scale_to_box(center, scale)
        
        return img, bbox
        

    def transform(self, src, bbox):
        x, y, w, h = bbox
        center, scale = _box_to_center_scale(
            x, y, w, h, self._aspect_ratio)
        scale = scale * 1.0

        input_size = self._input_size
        inp_h, inp_w = input_size

        trans = get_affine_transform(center, scale, 0, [inp_w, inp_h])
        img = cv2.warpAffine(src, trans, (int(inp_w), int(inp_h)), flags=cv2.INTER_LINEAR)
        bbox = _center_scale_to_box(center, scale)

        img = im_to_torch(img)
        img[0].add_(-0.406)
        img[1].add_(-0.457)
        img[2].add_(-0.480)

        return img, bbox

In [5]:
yolo_opt = EasyDict()
yolo_opt.gpus = [i for i in range(torch.cuda.device_count())]
yolo_opt.device = torch.device("cuda:" + str(yolo_opt.gpus[0]) if yolo_opt.gpus[0] >= 0 else "cpu")

In [7]:
model = TRTModule()
model.load_state_dict(torch.load('./pretrained_models/256x192_res50_lr1e-3_1x_reduced_TensorRT_FP16.pth'))
model.eval()
print('TensorRT network created.')

TRTModule()

In [10]:
detect = YOLO4TRTDetector(yolo_cfg, yolo_opt)
detect.load_model_trt('./pretrained_models/YOLO4-CSP_TensorRT_FP16.pth')

Loading YOLO4 model..
TensorRT network created


In [15]:
cfg = update_config('./configs/human_reduced/resnet/256x192_res50_lr1e-3_1x_reduced_new.yaml')
trans=Transform(cfg.DATA_PRESET.IMAGE_SIZE, cfg.DATA_PRESET.HEATMAP_SIZE)

In [17]:
heatmap_to_coord = get_func_heatmap_to_coord(cfg)
hm_size = cfg.DATA_PRESET.HEATMAP_SIZE
norm_type = cfg.LOSS.get('NORM_TYPE', None)

In [13]:
imgfolder ='/private/home/taoshaf/src/c2f-vol-demo/data/h36m/images/'
imgnames = os.listdir(imgfolder)
imgnames =[imgname for imgname in imgnames if imgname.endswith('.jpg')]

In [20]:
total = 0
res = dict()
with torch.no_grad():
    for (i, imgname) in enumerate(imgnames[:400]):
        imgfile = os.path.join(imgfolder, imgname)
        img = cv2.imread(imgfile)
        t1 = time_synchronized()
        bbox = detect.detect_one_frame(0,img)
        
        if not bbox:
            res[imgname[:-4]] = []
            print(imgname[:-4])
            continue
        bbox = bbox[0]['bbox']
        im, box = trans.transform(img[:,:,::-1], bbox)
        im=im.unsqueeze(0).cuda()
        output = model(im)
        t2 = time_synchronized()
        kpts = np.hstack(heatmap_to_coord(output[0],box,hm_shape=hm_size, norm_type=norm_type))
        total += t2-t1
        res[imgname[:-4]] = kpts.astype(np.float64)
        if (i+1)%100 == 0:
            print('[{}/{}] Average inference time: {} seconds'.format(i+1,i+1,total/(i+1)), end='\r')